In [1]:
from resnet import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np
import matplotlib.pyplot as plt

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device, 'torch.distributed.is_available():', torch.distributed.is_available())
print(device)
# torch.distributed.init_process_group()

cuda


In [2]:
# from lars.lars import LARS
head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))

# model = nn.parallel.DistributedDataParallel(ResNetCIFAR(head_g=head))
model = nn.DataParallel(ResNetCIFAR(head_g=head))

model = model.to(device)
batch_size = int(4096)

train(model, epochs=100, batch_size=batch_size, lr=0.3*batch_size/256, reg=1e-6, log_every_n=50)

# optimizer = LARS(model.parameters(), lr=0.1, momentum=0.9)
# optimizer.zero_grad()
# loss_fn(model(input), target).backward()
# optimizer.step()

device cuda
==> Preparing data..
Files already downloaded and verified
Adjusting learning rate of group 0 to 2.4000e-01.
Adjusting learning rate of group 0 to 2.4000e-01.

Epoch: 0


/home/zl310/ece661-hw-fa23/project_test/lars/lars.py:77: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /opt/conda/conda-bld/pytorch_1682343998658/work/torch/csrc/utils/python_arg_parser.cpp:1485.)
  d_p.add_(weight_decay, p.data)


Adjusting learning rate of group 0 to 6.9600e-01.
Test Loss=nan

Epoch: 1
[Step=50]	Loss=5.0805	2751.2 examples/second
Adjusting learning rate of group 0 to 1.1520e+00.
Test Loss=nan

Epoch: 2
Adjusting learning rate of group 0 to 1.6080e+00.
Test Loss=nan

Epoch: 3
[Step=100]	Loss=nan	2647.4 examples/second
Adjusting learning rate of group 0 to 2.0640e+00.
Test Loss=nan

Epoch: 4
Adjusting learning rate of group 0 to 2.5200e+00.
Test Loss=nan

Epoch: 5
[Step=150]	Loss=nan	2697.9 examples/second
Adjusting learning rate of group 0 to 2.9760e+00.
Test Loss=nan

Epoch: 6
Adjusting learning rate of group 0 to 3.4320e+00.
Test Loss=nan

Epoch: 7
[Step=200]	Loss=nan	2688.8 examples/second
Adjusting learning rate of group 0 to 3.8880e+00.
Test Loss=nan

Epoch: 8
Adjusting learning rate of group 0 to 4.3440e+00.
Test Loss=nan

Epoch: 9
[Step=250]	Loss=nan	2652.7 examples/second
Adjusting learning rate of group 0 to 4.8000e+00.
Test Loss=nan

Epoch: 10
Adjusting learning rate of group 0 to 4.79

Exception ignored in: <function _releaseLock at 0x7f2047355820>
Traceback (most recent call last):
  File "/home/zl310/anaconda3/envs/michael/lib/python3.8/logging/__init__.py", line 223, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


In [ ]:
# # b = torch.rand((64, 3, 28, 28))
# head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))

# model = ResNetCIFAR(head_g=head).to(device)
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transforms.ToTensor())
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=1)
# b = None
# for idx, (inputs, label) in enumerate(trainloader):
#     print(inputs.shape)
#     b = inputs.to(device)
#     break

# y = model(b)

# # xcs = F.cosine_similarity(y[None, :, :], y[:,None,:], dim=-1)
# # ce_loss = F.cross_entropy(xcs / temperature, target, reduction="mean")
# # diff_sum = 0
# # for rr, cc in enumerate(range(128)):
#     # diff_sum += xcs[rr, cc] - F.cosine_similarity(y[rr, :], y[cc, :], dim=-1)

torch.Size([128, 3, 32, 32])


tensor([1, 0, 3, 2, 5, 4, 7, 6])
